In [ ]:
import numpy as np
import librosa as lba
import soundfile as sf
import matplotlib.pyplot as plt
from random import random

In [ ]:
stimuli_folder = '../stimuli/'

sr = 44100
tone_duration = 200

rise_duration = 5
rise_length = int(rise_duration * sr / 1000)

fade_duration = 25
fade_length = int(fade_duration * sr / 1000)

tones = {
    'A7': 3520,
    'A6': 1760,
    'A5': 880,
    'A4': 440,
    'A3': 220,
    'A2': 110,
    'Ds5': 622.254
}

# Generate Tones

In [ ]:
dur = tone_duration / 1000
for pitch in tones:
    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    phase = random() * 2 * np.pi
    tone = lba.tone(f, sr=sr, duration=dur, phi=phase)
    
    # Add first three overtones with slope of -6 db (half amplitude) per octave
    for i in range(3):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=dur, phi=phase) / (i + 2)
    
    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()

    # Apply linear fade to ending
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)
    
    # Save tone
    sf.write('../stimuli/tones/tone%s.wav' % pitch, tone, 44100)

## (Balance tones in Audacity now, then return)

# Generate Sequences

In [ ]:
### Main Sequences ###

octaves = [2, 3, 4, 5, 6, 7]
iois = [1000, 918, 843,
        774, 710, 652,
        599, 550, 504,
        463, 425, 390,
        358, 329, 302]
for tone_type in (('piano', 'p'), ('tone', 'a')):
    for i, loudness in enumerate(('soft', 'normed', 'loud')):
        for octave in octaves:
            # Load tone of current octave
            tone, sr = lba.load(stimuli_folder + 'tones/%sA%i-%s.wav' % (tone_type[0], octave, loudness), sr=None)
            for ioi in iois:

                # Create array of appropriate length to hold audio sequence
                ms_ioi = ioi / 1000.
                sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

                # Insert tones at appropriate locations
                for j in range(5):
                    tone_start = int(np.ceil(j * ms_ioi * sr))
                    tone_end = tone_start + tone.shape[0]
                    sequence[tone_start:tone_end] = tone

                # Cut silence from the end of the sequence
                sequence = np.trim_zeros(sequence, 'b')

                # Save sequence to WAV file
                outfile = stimuli_folder + 'sequence_%s_%i_%i_%i.wav' % (tone_type[1], octave, ioi, i)
                sf.write(outfile, sequence, sr)

In [ ]:
### Practice Trials ###
for tone_type in (('piano', 'p'), ('tone', 'a')):
    tone, sr = lba.load(stimuli_folder + 'tones/%sDs5-normed.wav' % tone_type[0], sr=None)
    for ioi in (407, 550, 741):
        # Create array of appropriate length to hold audio sequence
        ms_ioi = ioi / 1000.
        sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

        # Insert tones at appropriate locations
        for i in range(5):
            tone_start = int(np.ceil(i * ms_ioi * sr))
            tone_end = tone_start + tone.shape[0]
            sequence[tone_start:tone_end] = tone

        # Cut silence from the end of the sequence
        sequence = np.trim_zeros(sequence, 'b')

        # Save sequence to WAV file
        outfile = stimuli_folder + 'practice_sequence_%i%s.wav' % (ioi, tone_type[1])
        sf.write(outfile, sequence, sr)

In [ ]:
### Metronome ###
tone, sr = lba.load(stimuli_folder + 'tones/tick.wav', sr=None)

# Create array of appropriate length to hold audio sequence
ioi = 550
ms_ioi = ioi / 1000.
sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)
        
# Insert tones at appropriate locations
for i in range(5):
    tone_start = int(np.ceil(i * ms_ioi * sr))
    tone_end = tone_start + tone.shape[0]
    sequence[tone_start:tone_end] = tone
        
# Cut silence from the end of the sequence
sequence = np.trim_zeros(sequence, 'b')
        
# Save sequence to WAV file
outfile = stimuli_folder + 'reference_sequence.wav'
sf.write(outfile, sequence, sr)